In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [2]:
df_exp=pd.read_csv('download_samples_tsv.tsv',sep='\t')

In [3]:
df_exp.columns

Index(['#sid', 'name', 'note', 'sample_env', 'keywords_clean', 'taxa_stats',
       'num_rids', 'num_hq_runs', 'rids', 'projects', 'publications'],
      dtype='object')

In [4]:
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724 entries, 0 to 723
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   #sid            724 non-null    object 
 1   name            724 non-null    object 
 2   note            0 non-null      float64
 3   sample_env      724 non-null    object 
 4   keywords_clean  724 non-null    object 
 5   taxa_stats      723 non-null    object 
 6   num_rids        724 non-null    int64  
 7   num_hq_runs     664 non-null    float64
 8   rids            724 non-null    object 
 9   projects        724 non-null    object 
 10  publications    724 non-null    object 
dtypes: float64(2), int64(1), object(8)
memory usage: 62.3+ KB


In [5]:
data = []
for _, row in df_exp.iterrows():  
    ers_id = row['#sid']
    keywords = row['keywords_clean']
    rids=row['rids']
    
    # Classify by specific allergen type
    if 'milk' in keywords or 'dairy food' in keywords:
        label = 'allergic'
        allergen_group = 'MILK'
    elif 'sesame' in keywords:
        label = 'allergic'
        allergen_group = 'SESAME'
    elif 'peanut' in keywords:
        label = 'allergic'
        allergen_group = 'PEANUT'
    elif 'walnut' in keywords or 'hazelnut' in keywords or 'nuts' in keywords:
        label = 'allergic'
        allergen_group = 'TREE_NUTS'
    elif 'egg' in keywords:
        label = 'allergic'
        allergen_group = 'EGG'
    else:
        # Samples without specific allergen keywords are controls
        label = 'healthy'
        allergen_group = 'CONTROL'
    
    data.append({
        'ERS_ID': ers_id, 
        'Label': label,
        'Allergen_Group': allergen_group,
        'rids':rids
    })

# Crea il nuovo dataframe
result_df_exp = pd.DataFrame(data)


In [6]:
result_df_exp.head(10)

,ERS_ID,Label,Allergen_Group,rids
0,ERS4516182,allergic,MILK,ERR4033678
1,ERS4516183,allergic,MILK,ERR4033679
2,ERS4516184,allergic,MILK,ERR4033680
3,ERS4516185,allergic,MILK,ERR4033681
4,ERS4516186,allergic,MILK,ERR4033682
5,ERS4516187,allergic,MILK,ERR4033683
6,ERS4516188,allergic,MILK,ERR4033684
7,ERS4516189,allergic,MILK,ERR4033685
8,ERS4516190,allergic,MILK,ERR4033686
9,ERS4516191,allergic,MILK,ERR4033687


In [7]:
result_df_exp.tail(10)

,ERS_ID,Label,Allergen_Group,rids
714,ERS4519304,allergic,SESAME,ERR4038127
715,ERS4519305,allergic,SESAME,ERR4038128
716,ERS4519306,allergic,SESAME,ERR4038129
717,ERS4519307,allergic,SESAME,ERR4038130
718,ERS4519308,allergic,SESAME,ERR4038131
719,ERS4519309,allergic,SESAME,ERR4038132
720,ERS4519310,allergic,SESAME,ERR4038133
721,ERS4519311,allergic,SESAME,ERR4038134
722,ERS4519312,allergic,SESAME,ERR4038135
723,ERS4519313,allergic,SESAME,ERR4038136


In [8]:
print(f"\nTotal samples: {len(result_df_exp)}")
print(f"\nGroup distribution:")
print(result_df_exp['Allergen_Group'].value_counts())


Total samples: 724

Group distribution:
Allergen_Group
MILK         322
SESAME       133
TREE_NUTS    129
PEANUT        55
CONTROL       51
EGG           34
Name: count, dtype: int64


In [9]:
# summary table
summary = result_df_exp['Allergen_Group'].value_counts().reset_index()
summary.columns = ['Allergen_Group', 'Count']
summary = summary.sort_values('Allergen_Group')
summary

,Allergen_Group,Count
4,CONTROL,51
5,EGG,34
0,MILK,322
3,PEANUT,55
1,SESAME,133
2,TREE_NUTS,129


In [10]:
# write csv file
result_df_exp_labels=result_df_exp.copy()
result_df_exp_labels.drop(['Allergen_Group'],axis=1, inplace=True)

In [11]:
result_df_exp_labels.head(10)

,ERS_ID,Label,rids
0,ERS4516182,allergic,ERR4033678
1,ERS4516183,allergic,ERR4033679
2,ERS4516184,allergic,ERR4033680
3,ERS4516185,allergic,ERR4033681
4,ERS4516186,allergic,ERR4033682
5,ERS4516187,allergic,ERR4033683
6,ERS4516188,allergic,ERR4033684
7,ERS4516189,allergic,ERR4033685
8,ERS4516190,allergic,ERR4033686
9,ERS4516191,allergic,ERR4033687


In [12]:
result_df_exp_labels.iloc[20:30]

,ERS_ID,Label,rids
20,ERS4516636,healthy,ERR4033809
21,ERS4516637,healthy,ERR4033810
22,ERS4516638,healthy,ERR4033811
23,ERS4516639,healthy,ERR4033812
24,ERS4516640,healthy,ERR4033813
25,ERS4516641,healthy,ERR4033814
26,ERS4516642,healthy,ERR4033815
27,ERS4516643,healthy,ERR4033816
28,ERS4516644,healthy,ERR4033817
29,ERS4516645,healthy,ERR4033818


In [13]:
result_df_exp_labels.tail(10)

,ERS_ID,Label,rids
714,ERS4519304,allergic,ERR4038127
715,ERS4519305,allergic,ERR4038128
716,ERS4519306,allergic,ERR4038129
717,ERS4519307,allergic,ERR4038130
718,ERS4519308,allergic,ERR4038131
719,ERS4519309,allergic,ERR4038132
720,ERS4519310,allergic,ERR4038133
721,ERS4519311,allergic,ERR4038134
722,ERS4519312,allergic,ERR4038135
723,ERS4519313,allergic,ERR4038136


In [14]:
result_df_exp_labels_renamed=result_df_exp_labels.replace({'Label':{'allergic':'1', 'healthy':'0'}})
result_df_exp_labels_renamed['Label'] = result_df_exp_labels_renamed['Label'].astype(int)
result_df_exp_labels_renamed=result_df_exp_labels_renamed[['ERS_ID','Label']]
result_df_exp_labels_renamed.head()

,ERS_ID,Label
0,ERS4516182,1
1,ERS4516183,1
2,ERS4516184,1
3,ERS4516185,1
4,ERS4516186,1


In [15]:
# result_df_exp_labels.to_csv('df_exp_labels.csv',index=False)
#result_df_exp_labels_renamed.to_csv('to_use/df_exp_all_labels.csv',index=False)

In [16]:
# add time 

In [17]:
df_run = pd.read_csv('SraRunTable.csv')

In [18]:
# time of trials from anonymized_name 
# For milk:   P1.milk.T1      -> T1
# For sesame: Sesame.T7.193   -> T7
# For nuts:   P2.Nuts.T1      -> T1
def extract_trial(name):
    # Look for something like T0, T1, T2, ..., T15 (case-insensitive)
    m = re.search(r'[Tt](\d+)', name)
    if m:
        # Always return normalized 'T#'
        return 'T' + m.group(1)
    else:
        # No T# found (e.g. "Con28"); replace with empty field
        return ''

# recompute the column
df_run['Trial'] = df_run['anonymized_name'].apply(extract_trial)

In [19]:
df_run[['Run', 'anonymized_name', 'Trial']].head()

,Run,anonymized_name,Trial
0,ERR4033678,P1.milk.T1,T1
1,ERR4033679,P1.milk.T2,T2
2,ERR4033680,P24.milk.T1,T1
3,ERR4033681,P24.milk.T2,T2
4,ERR4033682,P24.milk.T3,T3


In [20]:
df_run[['Run', 'anonymized_name', 'Trial']].tail()

,Run,anonymized_name,Trial
723,ERR4038132,Sesame.T7.193,T7
724,ERR4038133,Sesame.T7.47,T7
725,ERR4038134,Sesame.T8.48,T8
726,ERR4038135,Sesame.T9.194,T9
727,ERR4038136,Sesame.T9.50,T9


In [21]:
result_df_exp.head()

,ERS_ID,Label,Allergen_Group,rids
0,ERS4516182,allergic,MILK,ERR4033678
1,ERS4516183,allergic,MILK,ERR4033679
2,ERS4516184,allergic,MILK,ERR4033680
3,ERS4516185,allergic,MILK,ERR4033681
4,ERS4516186,allergic,MILK,ERR4033682


In [22]:
# JOIN 
result_df_exp = result_df_exp.rename(columns={'rids': 'Run'})
df_merged = result_df_exp.merge(df_run[['Run', 'anonymized_name','Trial']], on='Run', how='left')
result_time_df_exp=df_merged.copy()
result_time_df_exp_renamed=result_time_df_exp.replace({'Label':{'allergic':'1','healthy':'0'}})
result_time_df_exp_renamed['Label'] = result_time_df_exp_renamed['Label'].astype(int)                                                                
result_time_df_exp_renamed.head()

,ERS_ID,Label,Allergen_Group,Run,anonymized_name,Trial
0,ERS4516182,1,MILK,ERR4033678,P1.milk.T1,T1
1,ERS4516183,1,MILK,ERR4033679,P1.milk.T2,T2
2,ERS4516184,1,MILK,ERR4033680,P24.milk.T1,T1
3,ERS4516185,1,MILK,ERR4033681,P24.milk.T2,T2
4,ERS4516186,1,MILK,ERR4033682,P24.milk.T3,T3


In [23]:
result_time_df_exp_renamed.iloc[50:55]

,ERS_ID,Label,Allergen_Group,Run,anonymized_name,Trial
50,ERS4516666,0,CONTROL,ERR4033839,Con4,
51,ERS4516667,0,CONTROL,ERR4033840,Con5,
52,ERS4516668,0,CONTROL,ERR4033841,Con6,
53,ERS4516669,0,CONTROL,ERR4033842,Con7,
54,ERS4516670,0,CONTROL,ERR4033843,Con8,


In [24]:
result_time_df_exp_renamed.tail()

,ERS_ID,Label,Allergen_Group,Run,anonymized_name,Trial
719,ERS4519309,1,SESAME,ERR4038132,Sesame.T7.193,T7
720,ERS4519310,1,SESAME,ERR4038133,Sesame.T7.47,T7
721,ERS4519311,1,SESAME,ERR4038134,Sesame.T8.48,T8
722,ERS4519312,1,SESAME,ERR4038135,Sesame.T9.194,T9
723,ERS4519313,1,SESAME,ERR4038136,Sesame.T9.50,T9


In [25]:
result_time_df_exp_renamed.isna().sum()

ERS_ID             0
Label              0
Allergen_Group     0
Run                0
anonymized_name    0
Trial              0
dtype: int64

In [26]:
# 1) Extract patient_ID from anonymized_name
def extract_patient_id(name: str) -> str:
    s = str(name)
    # Case 1: starts with P + digits  (e.g. P1.milk.T1 -> P1)
    m = re.match(r'^(P\d+)', s)
    if m:
        return m.group(1)
    # Case 2: last part is numeric (e.g. Sesame.T7.193 -> 193)
    m2 = re.search(r'\.(\d+)$', s)
    if m2:
        return m2.group(1)
    # Fallback: return the full string if no clear pattern
    return s

result_time_df_exp_renamed['patient_ID'] = (
    result_time_df_exp_renamed['anonymized_name'].apply(extract_patient_id)
)




In [27]:
result_time_df_exp_renamed.head()

,ERS_ID,Label,Allergen_Group,Run,anonymized_name,Trial,patient_ID
0,ERS4516182,1,MILK,ERR4033678,P1.milk.T1,T1,P1
1,ERS4516183,1,MILK,ERR4033679,P1.milk.T2,T2,P1
2,ERS4516184,1,MILK,ERR4033680,P24.milk.T1,T1,P24
3,ERS4516185,1,MILK,ERR4033681,P24.milk.T2,T2,P24
4,ERS4516186,1,MILK,ERR4033682,P24.milk.T3,T3,P24


In [28]:
# Build dataset with the interested columns
base_df = result_time_df_exp_renamed[['Trial', 'ERS_ID', 'Label']].copy()
base_df.head()


,Trial,ERS_ID,Label
0,T1,ERS4516182,1
1,T2,ERS4516183,1
2,T1,ERS4516184,1
3,T2,ERS4516185,1
4,T3,ERS4516186,1


In [29]:
# Mask for controls (Label = 0)
controls = base_df[base_df['Label'] == 0].copy()

# Dataset for T1: only allergic samples from T1 + all controls, all labeled as T1
t1_allergic = base_df[(base_df['Trial'] == 'T1') & (base_df['Label'] == 1)].copy()
t1_allergic['Trial'] = 'T1'
controls_T1 = controls.copy()
controls_T1['Trial'] = 'T1'
df_T1 = pd.concat([t1_allergic, controls_T1], axis=0).reset_index(drop=True)

# Dataset for T2: only allergic samples from T2 + all controls, all labeled as T1
t2_allergic = base_df[(base_df['Trial'] == 'T2') & (base_df['Label'] == 1)].copy()
t2_allergic['Trial'] = 'T2'
controls_T2 = controls.copy()
controls_T2['Trial'] = 'T2'
df_T2 = pd.concat([t2_allergic, controls_T2], axis=0).reset_index(drop=True)

# Dataset for T3: only allergic samples from T3 + all controls, all labeled as T1
t3_allergic = base_df[(base_df['Trial'] == 'T3') & (base_df['Label'] == 1)].copy()
t3_allergic['Trial'] = 'T3'
controls_T3 = controls.copy()
controls_T3['Trial'] = 'T3'
df_T3 = pd.concat([t3_allergic, controls_T3], axis=0).reset_index(drop=True)




In [30]:
# Quick sanity checks
print("df_T1 shape:", df_T1.shape)
print("df_T2 shape:", df_T2.shape)
print("df_T3 shape:", df_T3.shape)


df_T1 shape: (261, 3)
df_T2 shape: (192, 3)
df_T3 shape: (160, 3)


In [31]:
#df_T1.to_csv("to_use/T1.csv", index=False)
#df_T2.to_csv("to_use/T2.csv", index=False)
#df_T3.to_csv("to_use/T3.csv", index=False)


In [32]:
df_T_merged=pd.concat([df_T1, df_T2,df_T3], axis=0).reset_index(drop=True)

In [33]:
df_T_merged

,Trial,ERS_ID,Label
0,T1,ERS4516182,1
1,T1,ERS4516184,1
2,T1,ERS4516187,1
3,T1,ERS4516188,1
4,T1,ERS4516191,1
...,...,...,...
608,T3,ERS4519207,0
609,T3,ERS4519208,0
610,T3,ERS4519209,0
611,T3,ERS4519210,0


In [34]:
df_allergic = df_T_merged[df_T_merged['Label'] != 0].reset_index(drop=True)

In [35]:
df_allergic

,Trial,ERS_ID,Label
0,T1,ERS4516182,1
1,T1,ERS4516184,1
2,T1,ERS4516187,1
3,T1,ERS4516188,1
4,T1,ERS4516191,1
...,...,...,...
455,T3,ERS4519277,1
456,T3,ERS4519278,1
457,T3,ERS4519279,1
458,T3,ERS4519280,1


In [36]:
result_time_df_exp_renamed

,ERS_ID,Label,Allergen_Group,Run,anonymized_name,Trial,patient_ID
0,ERS4516182,1,MILK,ERR4033678,P1.milk.T1,T1,P1
1,ERS4516183,1,MILK,ERR4033679,P1.milk.T2,T2,P1
2,ERS4516184,1,MILK,ERR4033680,P24.milk.T1,T1,P24
3,ERS4516185,1,MILK,ERR4033681,P24.milk.T2,T2,P24
4,ERS4516186,1,MILK,ERR4033682,P24.milk.T3,T3,P24
...,...,...,...,...,...,...,...
719,ERS4519309,1,SESAME,ERR4038132,Sesame.T7.193,T7,193
720,ERS4519310,1,SESAME,ERR4038133,Sesame.T7.47,T7,47
721,ERS4519311,1,SESAME,ERR4038134,Sesame.T8.48,T8,48
722,ERS4519312,1,SESAME,ERR4038135,Sesame.T9.194,T9,194


In [37]:
df_allergic_join=df_allergic.merge(result_time_df_exp_renamed, on=['ERS_ID', 'Trial','Label'], how='left')

In [38]:
df_allergic_join

,Trial,ERS_ID,Label,Allergen_Group,Run,anonymized_name,patient_ID
0,T1,ERS4516182,1,MILK,ERR4033678,P1.milk.T1,P1
1,T1,ERS4516184,1,MILK,ERR4033680,P24.milk.T1,P24
2,T1,ERS4516187,1,MILK,ERR4033683,P25.milk.T1,P25
3,T1,ERS4516188,1,MILK,ERR4033684,P26.milk.T1,P26
4,T1,ERS4516191,1,MILK,ERR4033687,P3.milk.T1,P3
...,...,...,...,...,...,...,...
455,T3,ERS4519277,1,SESAME,ERR4038100,Sesame.T3.592,592
456,T3,ERS4519278,1,SESAME,ERR4038101,Sesame.T3.609,609
457,T3,ERS4519279,1,SESAME,ERR4038102,Sesame.T3.621,621
458,T3,ERS4519280,1,SESAME,ERR4038103,Sesame.T3.87,87


In [39]:
df_allergic_join.loc[df_allergic_join['anonymized_name'].str.startswith('P1.', na=False) & df_allergic_join['Trial'].isin(['T1', 'T2', 'T3'])]

,Trial,ERS_ID,Label,Allergen_Group,Run,anonymized_name,patient_ID
0,T1,ERS4516182,1,MILK,ERR4033678,P1.milk.T1,P1
11,T1,ERS4516672,1,TREE_NUTS,ERR4033845,P1.Hazel.T1,P1
12,T1,ERS4516680,1,TREE_NUTS,ERR4033853,P1.Nuts.T1,P1
13,T1,ERS4516683,1,TREE_NUTS,ERR4033856,P1.Walnut.T1,P1
210,T2,ERS4516183,1,MILK,ERR4033679,P1.milk.T2,P1
217,T2,ERS4516673,1,TREE_NUTS,ERR4033846,P1.Hazel.T2,P1
218,T2,ERS4516684,1,TREE_NUTS,ERR4033857,P1.Walnut.T2,P1
353,T3,ERS4516674,1,TREE_NUTS,ERR4033847,P1.Hazel.T3,P1
354,T3,ERS4516675,1,MILK,ERR4033848,P1.Milk.T3,P1
355,T3,ERS4516681,1,TREE_NUTS,ERR4033854,P1.Nuts.T3,P1


In [40]:
# Sort by Trial first
df_allergic_join_sorted = df_allergic_join.sort_values('Trial')

# Create a temporary column with cleaned names (remove 'T*' pattern and convert to lowercase)
df_allergic_join_sorted['name_cleaned'] = df_allergic_join_sorted['anonymized_name'].str.replace(r'\.T\d+$', '', regex=True).str.lower()

# Drop duplicates based on the cleaned name, keeping the first occurrence (which will be the first Trial)
df_allergic_join_cleaned = df_allergic_join_sorted.drop_duplicates(subset='name_cleaned', keep='first')

# Remove the temporary column if you don't need it
df_allergic_join_cleaned = df_allergic_join_cleaned.drop(columns='name_cleaned')

# Reset the index
df_allergic_join_cleaned = df_allergic_join_cleaned.reset_index(drop=True)

In [41]:
df_allergic_join_cleaned

,Trial,ERS_ID,Label,Allergen_Group,Run,anonymized_name,patient_ID
0,T1,ERS4516182,1,MILK,ERR4033678,P1.milk.T1,P1
1,T1,ERS4519051,1,MILK,ERR4037874,Milk.T1.273,273
2,T1,ERS4519052,1,MILK,ERR4037875,Milk.T1.28,28
3,T1,ERS4519053,1,MILK,ERR4037876,Milk.T1.283,283
4,T1,ERS4519054,1,MILK,ERR4037877,Milk.T1.297,297
...,...,...,...,...,...,...,...
373,T3,ERS4518628,1,MILK,ERR4037614,Milk.T3.764,764
374,T3,ERS4518626,1,MILK,ERR4037612,Milk.T3.711,711
375,T3,ERS4516845,1,TREE_NUTS,ERR4034018,P9.Nuts.T3,P9
376,T3,ERS4518627,1,MILK,ERR4037613,Milk.T3.721,721


In [42]:
df_control=result_time_df_exp_renamed[result_time_df_exp_renamed['Allergen_Group']=='CONTROL']

In [43]:
df_control.shape

(51, 7)

In [44]:
df_u_merged=pd.concat([df_allergic_join_cleaned, df_control], axis=0).reset_index(drop=True)

In [45]:
df_u_merged

,Trial,ERS_ID,Label,Allergen_Group,Run,anonymized_name,patient_ID
0,T1,ERS4516182,1,MILK,ERR4033678,P1.milk.T1,P1
1,T1,ERS4519051,1,MILK,ERR4037874,Milk.T1.273,273
2,T1,ERS4519052,1,MILK,ERR4037875,Milk.T1.28,28
3,T1,ERS4519053,1,MILK,ERR4037876,Milk.T1.283,283
4,T1,ERS4519054,1,MILK,ERR4037877,Milk.T1.297,297
...,...,...,...,...,...,...,...
424,T0,ERS4519207,0,CONTROL,ERR4038030,Non.T0.673,673
425,T0,ERS4519208,0,CONTROL,ERR4038031,Non.T0.674,674
426,T0,ERS4519209,0,CONTROL,ERR4038032,Non.T0.675,675
427,T0,ERS4519210,0,CONTROL,ERR4038033,Non.T0.676,676


In [46]:
df_u_merged.to_csv("df_u_merged.csv", index=False)

In [47]:
df_U=df_u_merged[['ERS_ID','Label']].copy()
df_U

,ERS_ID,Label
0,ERS4516182,1
1,ERS4519051,1
2,ERS4519052,1
3,ERS4519053,1
4,ERS4519054,1
...,...,...
424,ERS4519207,0
425,ERS4519208,0
426,ERS4519209,0
427,ERS4519210,0


In [48]:
df_U.to_csv("to_use/U.csv", index=False)